# Sentinel-2 example notebook

This notebook showcases how to use  `PASEOS` to implement **Sentinel2-A (S2A)** and **Sentinel2-B (S2B)**. In particular, the notebook shows how to create `space_actors` orbiting as the **S2A** and **S2B** around Earth. In addition, it shows how to add `power` and `communication` devices. Furthermore, it demonstrates how to register activities to perform onboard data acquisition and processing to detect **volcanic eruptions** on `Sentinel-2 L1C data`. <br> **DISCLAIMER**: the notebook requires `rasterio` to run correctly, which is not included in the packets required to install `PASEOS`.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys 
import os
sys.path.insert(1, os.path.join("..",".."))
import pykep as pk
import numpy as np
import paseos
from paseos import ActorBuilder, SpacecraftActor
from utils import s2pix_detector, acquire_data
from paseos.utils.set_log_level import set_log_level
from paseos.utils.load_default_cfg import load_default_cfg
from paseos.visualization.space_animation import SpaceAnimation
import asyncio
set_log_level("INFO")


In [ ]:
# Define central body
earth = pk.planet.jpl_lp("earth")
# Defining today date as pk.epoch (27 - 10 - 22)
today=pk.epoch(8335.5, "mjd2000")

# 1) - Instantiate Sentinel 2 space actors

First of all, let's create the scaffolds for **S2A** and **S2B**. The scaffolds are objects which are not equipped with any `communication_device` nor `power_devices`. Furthermore, they neither have any `orbit`. 

In [ ]:
S2A = ActorBuilder.get_actor_scaffold(
    "Sentinel2-A", SpacecraftActor, [0,0,0], today
)

# Define local actor
S2B = ActorBuilder.get_actor_scaffold(
    "Sentinel2-B", SpacecraftActor, [0,0,0], today
)

#### 1.a) - Add an orbit for S2A and S2B

Since **S2A** and **S2B** are orbiting around Earth, let's define `earth` as `pykep.planet` object.

In [ ]:
# Define central body
earth = pk.planet.jpl_lp("earth")


To find a realistic orbits for **S2A** and **S2B**, we can exploit `Two Line Elements (TLEs)` (Downloaded on 27-10-2022). This would allow finding their ephemerides at time = 27-10-2022 12:00:00.

In [ ]:
#Define today (27-10-22)
today = pk.epoch(8335.5,"mjd2000")

sentinel2A_line1 = "1 40697U 15028A   22300.01175178 -.00001065  00000+0 -38995-3 0  9999"
sentinel2A_line2 = "2 40697  98.5650  12.8880 0001080  78.8662 281.2690 14.30806819383668"
sentinel2A = pk.planet.tle(sentinel2A_line1, sentinel2A_line2)

#Calculating S2A ephemerides.
sentinel2A_eph=sentinel2A.eph(today)

sentinel2B_line1 = "1 42063U 17013A   22300.18652110  .00000099  00000+0  54271-4 0  9998"
sentinel2B_line2 = "2 42063  98.5693  13.0364 0001083 104.3232 255.8080 14.30819357294601"
sentinel2B = pk.planet.tle(sentinel2B_line1, sentinel2B_line2)

#Calculating S2B ephemerides.
sentinel2B_eph=sentinel2B.eph(today)


**S2A** and **S2B** are always in phase opposition. The next cell show the positions of the two satellites in different instants (`2022-Oct-27 12:00:00`, `blue`) and (`2022-Oct-27 12:19:59`, `red`). 

In [ ]:
u, v = np.mgrid[0:2 * np.pi:30j, 0:np.pi:30j]
x = pk.EARTH_RADIUS * np.cos(u) * np.sin(v)
y = pk.EARTH_RADIUS * np.sin(u) * np.sin(v)
z = pk.EARTH_RADIUS * np.cos(v)
ax=pk.orbit_plots.plot_planet(sentinel2A,t0=today, color="b")
ax=pk.orbit_plots.plot_planet(sentinel2B,t0=today, axes=ax, color="b")
ax=pk.orbit_plots.plot_planet(sentinel2A,t0=pk.epoch(today.mjd2000 + 60 * 20 * pk.SEC2DAY), axes=ax, color="r")
ax=pk.orbit_plots.plot_planet(sentinel2B,t0=pk.epoch(today.mjd2000 + 60 * 20 * pk.SEC2DAY), axes=ax, color="r")
ax.plot_surface(x,y,z,alpha=0.1)

Since **S2A** and **S2B** are always in phase opposition, any communication between the satellites is impossible. Therefore, we will now set the **S2B** space actor orbit by inverting the direction of its speed. 

In [ ]:
#Adding orbits around Earth based on previously calculated ephemerides
ActorBuilder.set_orbit(S2A, sentinel2A_eph[0], sentinel2A_eph[1], today, earth)

# To enable the communication between S2A and S2B, the velocity vector is multiplied by - 1 making the satellite to orbit with opposite direction.
ActorBuilder.set_orbit(S2B, sentinel2B_eph[0], [-sentinel2B_eph[1][0], -sentinel2B_eph[1][1],-sentinel2B_eph[1][2]], today, earth)

### 1.b) - Add communication and power devices

Adding power devices. Max battery level: 102 Ah @ 28V is 2.81 kWh (effect of aging is neglected).  To identify the charging rate, we are assuming to be dependent only by the power of solar panels. We are neglecting eventual charging rate limits due to battery technology. The solar-panels power is 2300 W at begin of life and 1730 W at end of life. <br> Please, refer to: [Copernicus: Sentinel-2](https://www.eoportal.org/satellite-missions/copernicus-sentinel-2#space-segment.)

In [ ]:
ActorBuilder.set_power_devices(S2A, battery_level_in_Ws=10080000, max_battery_level_in_Ws=10080000, charging_rate_in_W=1736)
ActorBuilder.set_power_devices(S2B, battery_level_in_Ws=10080000, max_battery_level_in_Ws=10080000, charging_rate_in_W=1860)


Adding communication devices. Here we assume Sentinel 2A and Sentinel 2B to have an Inter-Satellite-Link at 100 Mbps.

In [ ]:
ActorBuilder.add_comm_device(S2A, "isl_transmitter", 100000)
ActorBuilder.add_comm_device(S2B, "isl_transmitter", 100000)

# 2) - Instantiate PASEOS simulation

To instantiate `PASEOS`, we consider **S2A** as `local_actor`. The initial time is set to `today`.

In [ ]:
# init simulation
cfg=load_default_cfg()
cfg.sim.start_time=today.mjd2000 * pk.DAY2SEC
sim = paseos.init_sim(S2A, cfg)

We now add **S2B** to the simulation.

In [ ]:
sim.add_known_actor(S2B)

## 3) - Dealing with PASEOS activities 

Here we demonstrate how you can use register and perform activities with `PASEOS`. To this aim, we will use `fetection of volcanic eruptions` as use case, assuming it will be possible to run it onboard `S2A`. To this aim, we will use a simplified version of the algorithm `[1] Massimetti, Francesco, et al. ""Volcanic hot-spot detection using SENTINEL-2: a comparison with MODIS–MIROVA thermal data series."" Remote Sensing 12.5 (2020): 820.` The algorithm leverages on bands `B8A`, `B11` and `B12` and perform pixel-level detection. <br> We will assume S2A will acquire images and then process them by using [1]. In this notebook, we assume to be able to detect the following images regardless their proper geografical position. We thanks the `ESA` project `END2END` for providing images and the implementation of [1].

### 3.a) - Registering activities 

The next lines will show how to register an activity to simulate data acquisition on **S2A**. <Br>Each activity is bounded to an `asynchronous functions`.  Let's define the asynchornous function `acquire_data_async()´ as follow.

In [ ]:
async def acquire_data_async(args):
    args[1][0]=acquire_data(args[0])
    await asyncio.sleep(3.4) #Acquisition for an L0 granule takes 3.6 seconds for S2A. 

The function `acquire_data_async()` uses `args[0]` as a filename to load the file and saves the acquired image into `args[1][0]`. Please, notice that using `args[1][0]` instead of `args[1]` is just a trick to be sure the acquired data are modifies the input argument `args[1]` by reference without creating a local copy of it. 
Therefore, let's define a placeholder `data_acquired` as a list containing 3 lists with `None` elements, which will be overwritted by `acquire_data_async()`.

In [ ]:
data_acquired=[[None], [None], [None]]

The next line will register the activity `data_acquisition` by associating int to the `activity_function` previously defined, `acquire_data_async`. It is up to the user to specify the power consumption in W for the activity. For Sentinel-2, the peak power of the imager is 266 W as reported in [Copernicus: Sentinel-2](https://www.eoportal.org/satellite-missions/copernicus-sentinel-2#space-segment.)

In [ ]:
# Register an activity that emulate data acquisition
sim.register_activity(
    "data_acquisition", activity_function=acquire_data_async, power_consumption_in_watt=266
)


Simlarly, we can register an activity to perform the volcanic event detection. 

In [ ]:
#Output event mask
output_event_masks=[[None], [None], [None]]

async def detect_volcanic_eruptions_async(args):
    args[1][0]=s2pix_detector(args[0][0])
    await asyncio.sleep(0.1)

# Register an activity that emulate event detection
sim.register_activity(
    "volcanic_event_detection", activity_function=detect_volcanic_eruptions_async, power_consumption_in_watt=10
)


### 3.b) - Performing activities.

We can now perform the activities. In scheduling the activities, we assume that data are acquired and stored into the mass memory to be, then, processed during the off part of the satellite duty cycle. Please, refer to: [Copernicus: Sentinel-2](https://www.eoportal.org/satellite-missions/copernicus-sentinel-2#space-segment.) Please note that, when `sim.perform_activity(...)` is called, the keyword `await` is needed  **only for Jupyter notebooks** to ensure the correct management of asynchronous functions. **Remove it otherwise**.

In [ ]:
data_to_acquire=["Etna_00.tif", "La_Palma_02.tif", "Mayon_02.tif"]

#Data acquisition
for n, data_name in zip(range(len(data_to_acquire)), data_to_acquire):
    #Run the activity
    await sim.perform_activity("data_acquisition", activity_func_args=[data_name, data_acquired[n]])


#Data processing
for n in range(len(data_to_acquire)):
    #Run the activity
    await sim.perform_activity("volcanic_event_detection", activity_func_args=[data_acquired[n],output_event_masks[n]])


### 3.c) - Results visualization

In [ ]:
import matplotlib.pyplot as plt
fig, ax=plt.subplots(1,3,figsize=(20,20))

for n in range(3):
    ax[n].imshow(data_acquired[n][0])
    ax[n].contour(output_event_masks[n][0], colors="r")

## 4) - PASEOS Animations

The next cells show how to use `PASEOS` animations. 

In [ ]:
from IPython import display
from loguru import logger
import time 

logger.remove() 
logger.add(sys.stderr, level="INFO")# using DEBUG level will create tons of text

anim = SpaceAnimation(sim)

dt = 100  # time step in seconds
for x in range(100):
    anim.animate(sim, dt)
    display.display(anim.fig) 
    time.sleep(0.01)
    display.clear_output(wait=True)